In [1]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
import numpy as np
from PIL import Image
import torch
import pandas as pd
import os
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split

In [2]:
# 图片和数据文件的路径
image_dir = r"E:\picturedataplussssss1/"
train_data_file = r"E:\pic_diffusion_vae/train_data.xlsx"
test_data_file = r"E:\pic_diffusion_vae/test_data.xlsx"

# 读取 Excel 文件
train_df = pd.read_excel(train_data_file, nrows=13000)
test_df = pd.read_excel(test_data_file, nrows=3500)

class CustomDataset(Dataset):
    def __init__(self, images_dir, dataframe):
        self.images_dir = images_dir
        self.dataframe = dataframe
        self.transform = transforms.Compose([
            transforms.Resize(224, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        
        
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = f"lcy{self.dataframe.iloc[idx, 0]}.jpg"
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        conditions = torch.tensor(self.dataframe.iloc[idx, 3:4].values, dtype=torch.float)

        
        return {'pixel_values': image, 'input_ids': conditions}

# 创建数据集实例
#dataset = CustomDataset(image_dir, data_df)




train_dataset = CustomDataset(image_dir, train_df)
test_dataset = CustomDataset(image_dir, test_df)


# 设置设备（CPU 或 GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 创建 DataLoader
#dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)
# 获取并打印第一批数据的形状
#for batch in dataloader:
#    pixel_values = batch['pixel_values']
#    input_ids = batch['input_ids']
#    print("Batch pixel_values shape:", pixel_values.shape)
#    print("Batch input_ids shape:", input_ids)
#    break

In [3]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import r2_score
# 定义 collate_fn 函数
def collate_fn(data):
    pixel_values = [i['pixel_values'] for i in data]
    input_ids = [i['input_ids'] for i in data]

    # 将列表的数据堆叠成一个新的Tensor，并转移到设备上
    pixel_values = torch.stack(pixel_values).to(device)
    input_ids = torch.stack(input_ids).to(device)
    
    return {'pixel_values': pixel_values, 'input_ids': input_ids}

# 创建 DataLoader 实例
train_loader = DataLoader(train_dataset, shuffle=True, collate_fn=collate_fn, batch_size=64, num_workers=0)
test_loader = DataLoader(test_dataset, shuffle=False, collate_fn=collate_fn, batch_size=64, num_workers=0)

# 创建 DataLoader 实例
#loader = DataLoader(dataset,
#                    shuffle=True,
#                    collate_fn=collate_fn,
#                    batch_size=32,
#                    num_workers=0)  # 可以根据实际情况调整 num_workers|

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F



import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(kernel_size=2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.AvgPool2d(kernel_size=2)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.AvgPool2d(kernel_size=2)

        self.flatten = nn.Flatten()

        # 计算池化后的尺寸（假设输入图像尺寸为224x224）
        # 经过五次2x2的池化操作后，图像尺寸为 (224 / 2 / 2 / 2 / 2 / 2) = 7
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)

        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool5(x)

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# 初始化模型
model = CNN()


In [5]:
model.requires_grad_(True)
model.train()
model.to(device)

D:\anaconda3\lib\site-packages\torch\nn\modules\module.py:1150: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:30.)
  return t.to(device, dtype if t.is_floating_point() or t.is_complex() else None, non_blocking)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d

In [6]:
# 定义损失函数
criterion = nn.MSELoss()

# 使用优化器中的weight_decay参数实现L2正则化
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.2)  # weight_decay用于L2正则化

In [7]:

# 训练函数
def train(model, criterion, optimizer, train_loader, test_loader, epochs=100, patience=40, r2_threshold=0.99, model_path="E:\pic_diffusion_vae\predictmodel/CNN_parametersC33Best.pth"):
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
    best_r2 = -np.inf
    best_epoch = -1
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch in train_loader:
            inputs, targets = batch['pixel_values'].to(device), batch['input_ids'].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss = loss
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()
        
        # 打印训练集上的损失、MAE和R²
        train_loss = running_loss / len(train_loader)
        train_mae = evaluate_mae(model, train_loader)
        train_r2 = calculate_r2(model, train_loader)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Train MAE: {train_mae}, Train R²: {train_r2}')

        # 测试集上的损失、MAE和R²
        test_loss = evaluate_loss(model, criterion, test_loader)
        test_mae = evaluate_mae(model, test_loader)
        test_r2 = calculate_r2(model, test_loader)
        print(f'Epoch {epoch+1}, Test Loss: {test_loss}, Test MAE: {test_mae}, Test R²: {test_r2}')

        # 学习率调度器
        scheduler.step(test_loss)

        
        # 保存最好的模型
        if test_r2 > best_r2:
            best_r2 = test_r2
            best_epoch = epoch
            torch.save(model.state_dict(), model_path)
            print(f'Saved Best Model at Epoch {epoch+1} with Test R²: {test_r2}')

        # 早停法
        if test_r2 >= r2_threshold:
            print(f'Early stopping at Epoch {epoch+1} with Test R²: {test_r2}')
            break

        # 早停法的patience
        if epoch - best_epoch > patience:
            print(f'Early stopping at Epoch {epoch+1} due to no improvement for {patience} epochs')
            break

In [8]:
def evaluate_mae(model, data_loader):
    model.eval()
    total_mae = 0.0
    with torch.no_grad():
        for batch in data_loader:
            inputs, targets = batch['pixel_values'].to(device), batch['input_ids'].to(device)

                
            outputs = model(inputs)
            mae = torch.mean(torch.abs(outputs - targets)/targets)
            total_mae += mae.item()
    return total_mae / len(data_loader)

In [9]:
# 评估损失函数
def evaluate_loss(model, criterion, data_loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in data_loader:
            inputs, targets = batch['pixel_values'].to(device), batch['input_ids'].to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(data_loader)

In [10]:
# 计算R²的函数
def calculate_r2(model, data_loader):
    model.eval()
    all_outputs = []
    all_targets = []
    with torch.no_grad():
        for batch in data_loader:
            inputs, targets = batch['pixel_values'].to(device), batch['input_ids'].to(device)
            outputs = model(inputs)
            all_outputs.append(outputs.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
    all_outputs = np.concatenate(all_outputs, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    return r2_score(all_targets, all_outputs)

In [ ]:
# 训练模型
train(model, criterion, optimizer, train_loader, test_loader, epochs=100)

D:\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1, Train Loss: 0.2843011670559645, Train MAE: 0.5650105233490467, Train R²: -0.5467519292199798
Epoch 1, Test Loss: 1.9633615505695343, Test MAE: 0.5667852056026459, Test R²: -0.5429012255055117
Saved Best Model at Epoch 1 with Test R²: -0.5429012255055117
Epoch 2, Train Loss: 0.23803466038778423, Train MAE: 0.45425194561481474, Train R²: 0.4242146714614453
Epoch 2, Test Loss: 0.7300956356525421, Test MAE: 0.4533817094564438, Test R²: 0.42621922129492207
Saved Best Model at Epoch 2 with Test R²: 0.42621922129492207
Epoch 3, Train Loss: 0.21417565075680614, Train MAE: 3.64252605676651, Train R²: -13.510277146753403
Epoch 3, Test Loss: 18.447879486083984, Test MAE: 3.5744146585464476, Test R²: -13.48896480234911
Epoch 4, Train Loss: 0.20592337675392627, Train MAE: 0.28280066899955275, Train R²: 0.6292930755552355
Epoch 4, Test Loss: 0.4647530156373978, Test MAE: 0.28503994733095167, Test R²: 0.6347173471821712
Saved Best Model at Epoch 4 with Test R²: 0.6347173471821712
Epoch 5, Tr

In [ ]:
# 保存整个模型
model_path = "E:\pic_diffusion_vae\predictmodel/CNN_modelr.pth"  # 替换为你的文件路径
torch.save(model.to('cpu'), model_path)
print(f"Model saved to {model_path}")

# 仅保存模型参数
parameters_path = "E:\pic_diffusion_vae\predictmodel/CNN_parametersr.pth"  # 替换为你的文件路径
torch.save(model.to('cpu').state_dict(), parameters_path)
print(f"Model parameters saved to {parameters_path}")

In [ ]:
import matplotlib.pyplot as plt
model.eval()

model.to(device)

true_values = []
predicted_values = []
print(len(test_loader))
# 遍历测试数据集并收集真实值和预测值
with torch.no_grad():
    for batch in test_loader:
        inputs, targets = batch['pixel_values'].to(device), batch['input_ids'].to(device)

        # 前向传播
        output = model(inputs)
        
        # 将输出从Tensor转换为numpy数组并收集
        predicted_values.extend(output.cpu().numpy())
        true_values.extend(targets.cpu().numpy())

# 绘制得分曲线
plt.figure(figsize=(10, 6))
plt.scatter(true_values, predicted_values, alpha=0.5, label='Predict vs True', edgecolors='k')
plt.plot([min(true_values), max(true_values)], [min(true_values), max(true_values)], 'r--', label='理想线')
plt.xlabel('True', fontsize=12)
plt.ylabel('Predict', fontsize=12)
plt.title('CNN_Predict', fontsize=15)
plt.legend()
plt.grid(True)

# 调整布局以确保标签显示完整
plt.tight_layout(pad=3.0)
plt.show()